In [1365]:
import numpy as np
import taichi as ti
import taichi.math as tm
from taichi.math import vec4, vec3, vec2
from PIL import Image

ti.init(arch=ti.gpu)

[Taichi] Starting on arch=metal


In [1366]:
dat1 = np.load("gs_npz/feng.png.npz")
dat2 = np.load("gs_npz/turing.png.npz")

In [1367]:
res = dat1["res"]
N = dat1["positions"].shape[0]

p1 = ti.field(vec2, shape=(N,))
r1 = ti.field(ti.f32, shape=(N,))
s1 = ti.field(vec2, shape=(N,))
c1 = ti.field(vec3, shape=(N,))
o1 = ti.field(ti.f32, shape=(N,))

p1.from_numpy(dat1["positions"])
r1.from_numpy(dat1["rotations"])
s1.from_numpy(dat1["scales"])
c1.from_numpy(dat1["colors"])
o1.from_numpy(dat1["opacities"])

In [1368]:
p2 = ti.field(vec2, shape=(N,))
r2 = ti.field(ti.f32, shape=(N,))
s2 = ti.field(vec2, shape=(N,))
c2 = ti.field(vec3, shape=(N,))
o2 = ti.field(ti.f32, shape=(N,))

p2.from_numpy(dat2["positions"])
r2.from_numpy(dat2["rotations"])
s2.from_numpy(dat2["scales"])
c2.from_numpy(dat2["colors"])
o2.from_numpy(dat2["opacities"])

In [1369]:
@ti.func
def gielis(
    phi: ti.f32, m: ti.f32, n1: ti.f32, n2: ti.f32, n3: ti.f32, a: ti.f32, b: ti.f32
) -> ti.f32:
    return tm.pow(
        tm.pow(ti.abs(tm.cos(m * phi / 4.0) / a), n2)
        + tm.pow(ti.abs(tm.sin(m * phi / 4.0) / b), n3),
        -1.0 / n1,
    )

In [ ]:
image = ti.field(vec3, shape=res)
transmissive = ti.field(ti.f32, shape=res)


@ti.kernel
def draw(k: ti.f32):
    for x, y in image:
        uv = 2.0 * (vec2(x, y) / vec2(res) - 0.5)
        for i in range(N):
            S1 = tm.exp(s1[i])
            R1 = r1[i]
            O1 = 1.0 / (1.0 + tm.exp(-o1[i]))
            C1 = 1.0 / (1.0 + tm.exp(-c1[i]))
            P1 = 2.0 / (1.0 + tm.exp(-p1[i])) - 1.0

            S2 = tm.exp(s2[i])
            R2 = r2[i]
            O2 = 1.0 / (1.0 + tm.exp(-o2[i]))
            C2 = 1.0 / (1.0 + tm.exp(-c2[i]))
            P2 = 2.0 / (1.0 + tm.exp(-p2[i])) - 1.0

            S = tm.mix(S1, S2, k)
            R = tm.mix(R1, R2, k)
            O = tm.mix(O1, O2, k)
            C = tm.mix(C1, C2, k)
            P = tm.mix(P1, P2, k)

            T = tm.mat2(
                tm.cos(R),
                tm.sin(R),
                -tm.sin(R),
                tm.cos(R),
            ) @ tm.mat2(S.x, 0, 0, S.y)
            T = tm.inverse(T)
            X = T @ (uv - P)
            # power = tm.exp(-0.5 * tm.dot(X, X))
            # radius = ti.sqrt(tm.dot(X, X))
            phi = ti.atan2(X.y, X.x)
            r = gielis(phi, 5, 2, 1, 1, 0.75, 0.75)

            # Fix: Use distance squared scaled by the Gielis radius
            dist_sq = tm.dot(X, X)
            power = tm.exp(-0.5 * dist_sq / (r * r))

            alpha = power * O
            alpha = tm.smoothstep(0.1, 0.3, alpha)
            image[x, y] += alpha * transmissive[x, y] * C
            transmissive[x, y] *= 1.0 - alpha

In [1371]:
def rasterize(k: float):
    image.fill(0.0)
    transmissive.fill(1.0)
    draw(k)
# rasterize(0.0)  
# Image.fromarray((np.clip(image.to_numpy(), 0.0, 1.0) * 255).astype(np.uint8))


for i in range(60):
    phi = i / 60 * np.pi * 2.0
    rasterize(np.cos(phi) * 0.5 + 0.5)
    Image.fromarray((np.clip(image.to_numpy(), 0.0, 1.0) * 255).astype(np.uint8)).save(f"outputs/{i}.png")

In [1372]:
# gui = ti.GUI("2dgs", res=tuple(res))
# time = 0.0
# while gui.running:
#     time += 0.05
#     rasterize(np.sin(time) * 0.5 + 0.5)
#     gui.set_image(image.to_numpy())
#     gui.show()

# gui.close()